In [2]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import numpy as np
import os
import scipy
import networkx as nx
from tqdm import tqdm, trange

# add root directory to be able to import packages
# todo: make all packages installable so they can be called/imported by environment
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

from utils.wheel import wheel
from utils.calcium import calcium
from utils.network import network
#from utils.animal_database import animal_database


Autosaving every 180 seconds


/tmp/ipykernel_1184248/65607256.py:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [39]:
################################################
############### NETWORK ANALYSIS ###############
################################################
data_dir = '/media/cat/4TB1/donato/DON-011735/22-10-26/calibration/suite2p/plane0/'

# initialize calcium object and load suite2p data
c = calcium.Calcium()
c.verbose = True                          # outputs additional information during processing
c.recompute_binarization = False           # recomputes binarization and other processing steps; False: loads from previous saved locations if avialable
c.data_dir = data_dir
c.load_suite2p()                          # this function assumes output dirs is either in data_dir OR data_dir/suite2p/plane0/
c.detrend_model_order = 1
c.save_python = True
c.save_matlab = False

#
c.load_binarization()
traces = c.F_upphase_bin    # c.F_upphase_bin

print ("binarized data (# cells, # time points): ", traces.shape)


data_dir2 = '/media/cat/4TB1/donato/DON-006085/20210523/002P-F/tif/suite2p/plane0/'

# initialize calcium object and load suite2p data
c2 = calcium.Calcium()
c2.verbose = True                          # outputs additional information during processing
c2.recompute_binarization = False           # recomputes binarization and other processing steps; False: loads from previous saved locations if avialable
c2.data_dir = data_dir2
c2.load_suite2p()                          # this function assumes output dirs is either in data_dir OR data_dir/suite2p/plane0/


#
c2.load_binarization()
traces = c2.F_upphase_bin    # c.F_upphase_bin

    

  Fluorescence data loading information
         sample rate:  30 hz
         self.F (fluorescence):  (38, 10001)
         self.Fneu (neuropile):  (38, 10001)
         self.iscell (Suite2p cell classifier output):  (944, 2)
              of which number of good cells:  (38,)
         self.spks (deconnvoved spikes):  (38, 10001)
         self.stat (footprints structure):  (38,)
         mean std over all cells :  78.5
   todo: print binarization defaults...
binarized data (# cells, # time points):  (38, 10001)
  Fluorescence data loading information
         sample rate:  30 hz
         self.F (fluorescence):  (1191, 55740)
         self.Fneu (neuropile):  (1191, 55740)
         self.iscell (Suite2p cell classifier output):  (2304, 2)
              of which number of good cells:  (1191,)
         self.spks (deconnvoved spikes):  (1191, 55740)
         self.stat (footprints structure):  (1191,)
         mean std over all cells :  25.0
   todo: print binarization defaults...


In [41]:
###############################################################
###############################################################
###############################################################

def find_peaks(upphase, F_raw):
    #print (upphase.shape)

    # find peak
    bursts = np.where(upphase>0)[0]
    #print (bursts)

    diffs = bursts[1:]-bursts[:-1]

    if diffs.shape[0]==0:
        return [], []
    
    ons = []
    start = bursts[0]
    end = 0
    for k in range(1,bursts.shape[0]-1,1):
        if diffs[k]>1:
            ons.append([start,bursts[k]])
            start = bursts[k+1]
            end = None
        #

    if end==None and start>bursts[k]:
        ons.append([start, bursts[k]])

    #
    f0 = np.median(F_raw)
        
    #
    peaks = []
    durations = []
    for burst in ons:
        if burst[1]>(burst[0]+1):
            try:
                temp = np.max(F_raw[burst[0]:burst[1]])

                #
                peaks.append(temp/f0)

                #
                durations.append(burst[1]-burst[0])
            except:
                pass
            
    
    return peaks, durations


# compute the peaks of each burst
fps = 30

cell_ids = np.arange(38)

all_peaks = []
all_durations = []
for cell_id in tqdm(cell_ids):
    peaks, durations = find_peaks(c.F_upphase_bin[cell_id], c.F[cell_id])
    
    all_peaks.append(peaks)
    all_durations.append(durations)

y1 = np.histogram(np.hstack(all_peaks), bins=np.arange(0,5,0.1))
y11 = np.histogram(np.hstack(all_durations), bins=np.arange(0,110,1))
    

############################################
all_peaks = []
all_durations = []
for cell_id in tqdm(cell_ids):
    peaks, durations = find_peaks(c2.F_upphase_bin[cell_id], c2.F[cell_id])
    
    all_peaks.append(peaks)
    all_durations.append(durations)

y2 = np.histogram(np.hstack(all_peaks), bins=np.arange(0,5,0.1))
y22 = np.histogram(np.hstack(all_durations), bins=np.arange(0,110,1))
    

#########################################################
plt.figure()
ax=plt.subplot(1,2,1)
plt.plot(y1[1][:-1], y1[0]/np.max(y1[0]), label='gcamp8')
plt.plot(y2[1][:-1], y2[0]/np.max(y2[0]), label='gcamp6')
plt.xlabel("SNR (max peak/f0)")
plt.legend()
#
ax=plt.subplot(1,2,2)
plt.plot(y11[1][:-1]/fps, y11[0]/np.max(y11[0]))
plt.plot(y22[1][:-1]/fps, y22[0]/np.max(y22[0]))
plt.xlabel("Duration of up phase (sec)")
plt.suptitle(data_dir, fontsize=10)

plt.show()

100%|██████████████████████████████████████████████████████████████████████████████████████████| 38/38 [00:00<00:00, 331.99it/s]


In [35]:
# 
cell_id = 200

#
t = np.arange(c2.F_onphase_bin.shape[1])/30.
plt.figure()
temp = c2.F_filtered[cell_id]
#temp = temp - np.median(temp)
#temp = temp / np.max(temp)
plt.plot(t,temp, label='F_filtered')

plt.plot(t,c2.F_upphase_bin[cell_id]*np.max(temp), 
         alpha=.5,label="upphase")

plt.legend()
plt.xlabel("Time (sec)")
plt.title(data_dir2 + ",   cell #"+str(cell_id))
plt.show()

In [34]:
#############################################
######### LOAD SUITE2P BINARIZED DATA #######
#############################################

# input directory where Suite2p outputs matlab Fall.mat and all .npy files
# data_dir = '/media/cat/4TB/donato/steffen/DON-004366/20210228/'
# data_dir = '/media/cat/4TB/donato/steffen/DON-004366/20210301/'
#data_dir = '/media/cat/4TB/donato/DON-003343/DON-003343_20210214/suite2p/plane0/'
# data_dir = '/media/cat/4TB/donato/nathalie/plane0'
# data_dir = '/media/cat/4TB/donato/renan/renan'
#data_dir = '/media/cat/4TB/donato/steffen/DON-004366/20210228/'
data_dir = '/media/cat/4TB/donato/DON-006084/20210520/'

data_dir = '/media/cat/4TB/donato/DON-006084/20210520/'

# initialize calcium object and load suite2p data
c = calcium.Calcium()
c.verbose = True                          # outputs additional information during processing
c.recompute_binarization = False           # recomputes binarization and other processing steps; False: loads from previous saved locations if avialable
c.data_dir = data_dir
c.load_suite2p()                          # this function assumes output dirs is either in data_dir OR data_dir/suite2p/plane0/


#
c.load_binarization()
traces = c.F_upphase_bin    # c.F_upphase_bin

print ("binarized data (# cells, # time points): ", traces.shape)




  Fluorescence data loading information
         sample rate:  30 hz
         self.F (fluorescence):  (584, 55740)
         self.Fneu (neuropile):  (584, 55740)
         self.iscell (Suite2p cell classifier output):  (717, 2)
              of which number of good cells:  (584,)
         self.spks (deconnvoved spikes):  (584, 55740)
         self.stat (footprints structure):  (584,)
         mean std over all cells :  22.5
   todo: print binarization defaults...
binarized data (# cells, # time points):  (584, 55740)


In [35]:
#################################################
####### OPTIONAL: REMOVE DUPLICATE CELLS ########
#################################################
c.load_footprints()
c.deduplication_method = 'overlap'      # 'overlap'; 'centre_distance'
c.corr_min_distance = 8                 # min distance for centre_distance method - NOT USED HERE
c.corr_max_percent_overlap = 0.25       # max normalized cell body allowed 
c.corr_threshold = 0.3                  # max correlation allowed for high overlap; 
                                        #     note correlations computed using filtered fluorescecne not binarized
c.corr_delete_method = 'lowest_snr'     # highest_connected: removes hub neurons,keeps more cells; 
                                        # lowest_snr - removes lower SNR cells, keep less neurons
c.recompute_deduplication = False        # recompute the dedplucaiton wif new paramters are saved
c.n_cores = 8                           #  Number of cores to use for multi processing

#
c.remove_duplicate_neurons()            

#       
traces_unique = traces[c.clean_cell_ids]
print ("All cells: ", c.F.shape, "  unique cells: ", traces_unique.shape)


 ... deduplicating cells... (can take ~5 for ~1,000cells) 


8001it [00:00, 97660.61it/s]                                                                                                                              

All cells:  (584, 55740)   unique cells:  (424, 55740)


In [36]:
###############################################################
### OPTIONAL: LOAD WHEEL DATA AND QUEISCENT OR RUN PERIODS ####
###############################################################
# 
w = wheel.Wheel()
w.root_dir = os.path.join(c.data_dir.replace('suite2p/','').replace('plane0',''),    # this assumes wheel data from Steffens' matlab scripts
                          'TRD-2P')                                                   #   is saved in the root diretory of the animal
w.load_track()
w.compute_velocity()
print ("Exp time : ", w.track.velocity.times.shape[0]/w.imaging_sample_rate)

# 
w.max_velocity_quiescent = 0.001  # in metres per second
idx_quiescent = w.get_indexes_quiescent_periods()

#
w.min_velocity_running = 0.1  # in metres per second
idx_run = w.get_indexes_run_periods()



Exp time :  1858.0
   Queiscent periods (seconds):  1359.2333333333333
   Running periods (seconds):  170.1


In [60]:
#########################################################
####### RUN PCA ON ALL OR SUBSET OF TRACES ##############
#########################################################
#

# take only 200 cells; and first 10 minutes
n_cells = 200
traces_unique = traces_unique[:n_cells]
print ("Selected cells (# cells, time points): ", traces_unique.shape)

# take quiescent periods
traces_in = traces_unique[:,idx_quiescent]

#
n_sec = 1300*30
times = np.arange(n_sec)
traces_in = traces_in[:,times]

#
suffix1 = 200   # # of cells
suffix2 = 600   # # seconds
recompute=True
pca, X_pca = c.compute_PCA(traces_in, suffix1,suffix2,recompute)

# plot  variance exaplined
var_exp = pca.explained_variance_ratio_
# 
fig=plt.figure()
ax=plt.subplot(111)
plt.plot(var_exp, label='variance explained')
plt.ylabel("variance explained")
plt.xlabel("PC component")
plt.ylim(0,0.05)
plt.legend()

ax2=ax.twinx()
cumsum = np.cumsum(var_exp)
cumsum = cumsum/np.max(cumsum)
plt.plot(cumsum, c='red', label='cumulative variance explained')
#plt.semilogy()
plt.legend()


plt.show()



Selected cells (# cells, time points):  (200, 55740)


In [3]:
# 
fig=plt.figure()
ax=plt.subplot(111)
plt.plot(var_exp, label='variance explained')
plt.ylabel("variance explained")
plt.xlabel("PC component")
#plt.ylim(0,0.05)
plt.legend()
plt.semilogy()

plt.show()


NameError: name 'var_exp' is not defined

In [29]:
import numpy as np
import cv2
size = 512,512
duration = 2
fps = 150

fname = '/media/cat/4TB/donato/andres/objective_test/video_gelobj/Image_001_001.raw'
#fname = '/media/cat/4TB/donato/andres/objective_test/video_airobj/Image_001_001.raw'
d = np.fromfile(fname,dtype=np.uint16)
d = d.reshape(-1,512,512)

# plt.figure()
# plt.imshow(d[100:120].mean(0))
# plt.show()

d = (d-d.min())/(d.max()-d.min())*1024

out = cv2.VideoWriter(fname[:-4]+".mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (size[1], size[0]), False)
bin = 10
for k in range(0,1000,1):
    #data = np.random.randint(0, 256, size, dtype='uint8')
    #data = d[k*20:k*20+20].mean(0)
    data = d[k:k+bin].mean(0)
    #data = (data-data.min())/(data.max()-data.min())*256
    data = data.astype('uint8')
    #cvuint8 = cv2.convertScaleAbs(data)
    #plt.imshow(d[100:120].mean(0))
    out.write(data)
    #break
out.release()